<a href="https://colab.research.google.com/github/tnstanHCMUS/TQHDL/blob/main/Profilling_Pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**<center>Đồ án thực hành môn trực quan hoá: DATA VISUALIZATION USING D3.JS</center>**

# Thông tin sinh viên

- Nhóm 10:
    + 22127163 - Trần Đan Huy
    + 22127290 - Nguyễn Thị Thu Ngân
    + 22127029 - Lê Nguyễn Gia Bảo
    + 22127026 - Ôn Gia Bảo

# Import

In [4]:
import pandas as pd
import numpy as np
import os

# Tải csv lên hệ thống


In [5]:
from google.colab import files
uploaded = files.upload()

Saving project_heart_disease.csv to project_heart_disease.csv


# Đọc dữ liệu

In [20]:
import pandas as pd
from decimal import Decimal, InvalidOperation

# Đọc file, tắt tự động chuyển giá trị trống thành NaN
df = pd.read_csv('project_heart_disease.csv', encoding='utf-8-sig', keep_default_na=False)

# Hàm chuyển về int nếu có thể
def to_int_or_keep(value):
    try:
        # Nếu giá trị là số thực dạng x.0, cũng ép thành int
        if isinstance(value, str) and value.replace('.', '', 1).isdigit():
            if '.' in value:
                float_val = float(value)
                if float_val.is_integer():
                    return int(float_val)
            return int(value)
        elif isinstance(value, (int, float)) and float(value).is_integer():
            return int(value)
        else:
            return value
    except (ValueError, TypeError):
        return value

# Hàm chuyển về Decimal nếu có thể
def to_decimal_or_keep(value):
    try:
        return Decimal(value)
    except (InvalidOperation, ValueError, TypeError):
        return value

# Xử lý từng cột
df['Age'] = df['Age'].apply(to_int_or_keep)
df['Blood Pressure'] = df['Blood Pressure'].apply(to_int_or_keep)
df['Cholesterol Level'] = df['Cholesterol Level'].apply(to_int_or_keep)
df['BMI'] = df['BMI'].apply(to_decimal_or_keep)

# Kiểm tra kết quả
print(df[['Age', 'Blood Pressure', 'Cholesterol Level', 'BMI']].head())


  Age Blood Pressure Cholesterol Level                 BMI
0  56            153               155  24.991591091690363
1  69            146               286    25.2217985244363
2  46            126               216   29.85544714237164
3  32            122               293  24.130476882852445
4  60            166               242   20.48628889057518


#Data profilling

###1. Overview

In [34]:
# Tổng số dòng
total_rows = len(df)

# Hàm phân tích chất lượng cột
def column_quality(col):
    total = len(df)
    nulls = df[col].isnull().sum()
    missing = df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    actual = total - nulls - missing
    completeness = actual / total if total > 0 else 0

    # Chỉ lấy các giá trị hợp lệ (không NaN, không chuỗi rỗng)
    valid_values = df[col][(df[col].notna()) & (df[col].astype(str).str.strip() != '')]
    cardinality = valid_values.nunique()

    # Tính uniqueness và distinctness chuẩn theo định nghĩa
    uniqueness = cardinality / total if total > 0 else 0
    distinctness = cardinality / actual if actual > 0 else 0

    dtype = df[col].dtype

    return pd.Series({
        'Data Type': dtype,
        'Total Records': total,
        'NULL Count': nulls,
        'Missing Count': missing,
        'Actual Count': actual,
        'Completeness (%)': f"{completeness * 100:.2f}%",
        'Cardinality': cardinality,
        'Uniqueness (%)': f"{uniqueness * 100:.2f}%",
        'Distinctness (%)': f"{distinctness * 100:.2f}%"
    })

# Áp dụng cho toàn bộ cột
quality_df = df.columns.to_series().apply(column_quality)

# Định dạng lại bảng
quality_df = quality_df.reset_index().rename(columns={'index': 'Column'})
quality_df.index += 1  # STT bắt đầu từ 1

quality_df

,Column,Data Type,Total Records,NULL Count,Missing Count,Actual Count,Completeness (%),Cardinality,Uniqueness (%),Distinctness (%)
1,Age,object,10000,0,29,9971,99.71%,63,0.63%,0.63%
2,Gender,object,10000,0,19,9981,99.81%,2,0.02%,0.02%
3,Blood Pressure,object,10000,0,19,9981,99.81%,61,0.61%,0.61%
4,Cholesterol Level,object,10000,0,30,9970,99.70%,151,1.51%,1.51%
5,Exercise Habits,object,10000,0,25,9975,99.75%,3,0.03%,0.03%
6,Smoking,object,10000,0,25,9975,99.75%,2,0.02%,0.02%
7,Family Heart Disease,object,10000,0,21,9979,99.79%,2,0.02%,0.02%
8,BMI,object,10000,0,22,9978,99.78%,9978,99.78%,100.00%
9,Alcohol Consumption,object,10000,0,2586,7414,74.14%,3,0.03%,0.04%
10,Stress Level,object,10000,0,22,9978,99.78%,3,0.03%,0.03%


###2. Details

In [59]:
def analyze_column(df, column):
    from termcolor import colored
    from IPython.display import Markdown, display
    import pandas as pd

    col_data = df[column]
    total = len(col_data)

    # Lọc giá trị hợp lệ: Không NaN và không chuỗi rỗng
    valid_values = col_data[(col_data.notna()) & (col_data.astype(str).str.strip() != '')]

    # Tính các chỉ số
    nulls = col_data.isnull().sum()
    missing = (col_data.astype(str).str.strip() == '').sum()
    actual = total - nulls - missing

    completeness = actual / total if total != 0 else 0
    cardinality = valid_values.nunique()
    uniqueness = cardinality / total if total != 0 else 0
    distinctness = cardinality / actual if actual != 0 else 0

    min_len = valid_values.astype(str).apply(len).min() if actual != 0 else None
    max_len = valid_values.astype(str).apply(len).max() if actual != 0 else None

    data_type = valid_values.map(type).mode()[0].__name__ if actual > 0 else 'unknown'
    sample_val = str(valid_values.astype(str).iloc[0]) if actual > 0 else ''
    field_format = ''.join(['n' if c.isdigit() else 'x' for c in sample_val]) if sample_val else ''

    top5 = col_data.value_counts(dropna=False).head(5)

    # Hàm phụ để in đẹp
    def print_section(title):
        display(Markdown(f"### <span style='color:#2e6cbb'>{title}</span>"))

    def print_kv(label, value):
        print(f"{label:<32}: {value}")
        print("-" * 50)

    # In thông tin
    print_section(f"🔎 INPUT METADATA — {column.upper()}")
    print_kv("Field Name", column)
    print_kv("Field Data Type", data_type.upper())
    print_kv("Field Format", field_format if field_format else "N/A")
    print_kv("Field Length (MAX)", max_len if max_len is not None else "N/A")

    print_section("📊 DATA PROFILING SUMMARY STATISTICS")
    print_kv("Total Rows", total)
    print_kv("NULL Count", nulls)
    print_kv("Missing (blank strings)", missing)
    print_kv("Actual (Non-missing)", actual)
    print_kv("Completeness (%)", f"{completeness*100:.2f}%")
    print_kv("Cardinality", cardinality)
    print_kv("Uniqueness (%)", f"{uniqueness*100:.2f}%")
    print_kv("Distinctness (%)", f"{distinctness*100:.2f}%")

    print_section("📐 DATA PROFILING ADDITIONAL STATISTICS")
    print_kv("Field Data Types (Count)", valid_values.map(type).nunique())
    print_kv("Field Length (MIN)", min_len if min_len is not None else "N/A")
    print_kv("Field Length (MAX)", max_len if max_len is not None else "N/A")

    # Nếu có ít nhất 1 số hợp lệ thì in MIN/MAX Value
    try:
        numeric_col = pd.to_numeric(valid_values.astype(str), errors='coerce')
        if numeric_col.notnull().any():
            print_kv("Field Value (MIN)", numeric_col.min())
            print_kv("Field Value (MAX)", numeric_col.max())
    except Exception as e:
        pass

    print_section(f"⭐ {column.upper()} — TOP 5 VALUES")
    print(f"{'Value':<30}{'Count':>10}{'Percentage':>12}")
    print("-" * 55)
    for val, count in top5.items():
        percent = round(count / total * 100, 2)
        val_str = str(val)[:30]
        print(f"{val_str:<30}{count:>10}{percent:>12}%")


####2.1. Age

In [60]:
#viết hàm thực thi cho thuộc tính age
analyze_column(df, 'Age')


### <span style='color:#2e6cbb'>🔎 INPUT METADATA — AGE</span>

Field Name                      : Age
--------------------------------------------------
Field Data Type                 : INT
--------------------------------------------------
Field Format                    : nn
--------------------------------------------------
Field Length (MAX)              : 2
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 29
--------------------------------------------------
Actual (Non-missing)            : 9971
--------------------------------------------------
Completeness (%)                : 99.71%
--------------------------------------------------
Cardinality                     : 63
--------------------------------------------------
Uniqueness (%)                  : 0.63%
--------------------------------------------------
Distinctness (%)                : 0.63%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 2
--------------------------------------------------
Field Length (MAX)              : 2
--------------------------------------------------
Field Value (MIN)               : 18
--------------------------------------------------
Field Value (MAX)               : 80
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ AGE — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
71                                   187        1.87%
64                                   185        1.85%
43                                   182        1.82%
34                                   182        1.82%
62                                   181        1.81%


####2.2. gender

In [61]:
analyze_column(df, 'Gender')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — GENDER</span>

Field Name                      : Gender
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxx
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 19
--------------------------------------------------
Actual (Non-missing)            : 9981
--------------------------------------------------
Completeness (%)                : 99.81%
--------------------------------------------------
Cardinality                     : 2
--------------------------------------------------
Uniqueness (%)                  : 0.02%
--------------------------------------------------
Distinctness (%)                : 0.02%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 4
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ GENDER — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Male                                5003       50.03%
Female                              4978       49.78%
                                      19        0.19%


####2.3. Blood Pressure

In [62]:
analyze_column(df, 'Blood Pressure')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — BLOOD PRESSURE</span>

Field Name                      : Blood Pressure
--------------------------------------------------
Field Data Type                 : INT
--------------------------------------------------
Field Format                    : nnn
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 19
--------------------------------------------------
Actual (Non-missing)            : 9981
--------------------------------------------------
Completeness (%)                : 99.81%
--------------------------------------------------
Cardinality                     : 61
--------------------------------------------------
Uniqueness (%)                  : 0.61%
--------------------------------------------------
Distinctness (%)                : 0.61%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 3
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------
Field Value (MIN)               : 120
--------------------------------------------------
Field Value (MAX)               : 180
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ BLOOD PRESSURE — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
134                                  214        2.14%
167                                  195        1.95%
171                                  182        1.82%
170                                  181        1.81%
142                                  181        1.81%


####2.4. Cholesterol Level

In [63]:
analyze_column(df, 'Cholesterol Level')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — CHOLESTEROL LEVEL</span>

Field Name                      : Cholesterol Level
--------------------------------------------------
Field Data Type                 : INT
--------------------------------------------------
Field Format                    : nnn
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 30
--------------------------------------------------
Actual (Non-missing)            : 9970
--------------------------------------------------
Completeness (%)                : 99.70%
--------------------------------------------------
Cardinality                     : 151
--------------------------------------------------
Uniqueness (%)                  : 1.51%
--------------------------------------------------
Distinctness (%)                : 1.51%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 3
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------
Field Value (MIN)               : 150
--------------------------------------------------
Field Value (MAX)               : 300
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ CHOLESTEROL LEVEL — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
292                                   91        0.91%
186                                   84        0.84%
185                                   83        0.83%
288                                   83        0.83%
193                                   81        0.81%


####2.5 Exercise Habits

In [64]:
analyze_column(df, 'Exercise Habits')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — EXERCISE HABITS</span>

Field Name                      : Exercise Habits
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxx
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 25
--------------------------------------------------
Actual (Non-missing)            : 9975
--------------------------------------------------
Completeness (%)                : 99.75%
--------------------------------------------------
Cardinality                     : 3
--------------------------------------------------
Uniqueness (%)                  : 0.03%
--------------------------------------------------
Distinctness (%)                : 0.03%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 3
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ EXERCISE HABITS — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
High                                3372       33.72%
Medium                              3332       33.32%
Low                                 3271       32.71%
                                      25        0.25%


####2.6 Smoking

In [65]:
analyze_column(df, 'Smoking')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — SMOKING</span>

Field Name                      : Smoking
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxx
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 25
--------------------------------------------------
Actual (Non-missing)            : 9975
--------------------------------------------------
Completeness (%)                : 99.75%
--------------------------------------------------
Cardinality                     : 2
--------------------------------------------------
Uniqueness (%)                  : 0.02%
--------------------------------------------------
Distinctness (%)                : 0.02%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 2
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ SMOKING — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Yes                                 5123       51.23%
No                                  4852       48.52%
                                      25        0.25%


####2.7 Family Heart Disease

In [66]:
analyze_column(df, 'Family Heart Disease')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — FAMILY HEART DISEASE</span>

Field Name                      : Family Heart Disease
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxx
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 21
--------------------------------------------------
Actual (Non-missing)            : 9979
--------------------------------------------------
Completeness (%)                : 99.79%
--------------------------------------------------
Cardinality                     : 2
--------------------------------------------------
Uniqueness (%)                  : 0.02%
--------------------------------------------------
Distinctness (%)                : 0.02%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 2
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ FAMILY HEART DISEASE — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
No                                  5004       50.04%
Yes                                 4975       49.75%
                                      21        0.21%


####2.8 BMI

In [67]:
analyze_column(df, 'BMI')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — BMI</span>

Field Name                      : BMI
--------------------------------------------------
Field Data Type                 : DECIMAL
--------------------------------------------------
Field Format                    : nnxnnnnnnnnnnnnnnn
--------------------------------------------------
Field Length (MAX)              : 18
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 22
--------------------------------------------------
Actual (Non-missing)            : 9978
--------------------------------------------------
Completeness (%)                : 99.78%
--------------------------------------------------
Cardinality                     : 9978
--------------------------------------------------
Uniqueness (%)                  : 99.78%
--------------------------------------------------
Distinctness (%)                : 100.00%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 14
--------------------------------------------------
Field Length (MAX)              : 18
--------------------------------------------------
Field Value (MIN)               : 18.00283694367505
--------------------------------------------------
Field Value (MAX)               : 39.99695379658124
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ BMI — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
                                      22        0.22%
29.667206440832203                     1        0.01%
22.649772128045168                     1        0.01%
28.736666746435628                     1        0.01%
27.840845436728944                     1        0.01%


####2.9 Alcohol Consumption

In [68]:
analyze_column(df, 'Alcohol Consumption')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — ALCOHOL CONSUMPTION</span>

Field Name                      : Alcohol Consumption
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxx
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 2586
--------------------------------------------------
Actual (Non-missing)            : 7414
--------------------------------------------------
Completeness (%)                : 74.14%
--------------------------------------------------
Cardinality                     : 3
--------------------------------------------------
Uniqueness (%)                  : 0.03%
--------------------------------------------------
Distinctness (%)                : 0.04%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 3
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ ALCOHOL CONSUMPTION — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
                                    2586       25.86%
Medium                              2500        25.0%
Low                                 2488       24.88%
High                                2426       24.26%


####2.10 Stress Level

In [69]:
analyze_column(df, 'Stress Level')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — STRESS LEVEL</span>

Field Name                      : Stress Level
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxx
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 22
--------------------------------------------------
Actual (Non-missing)            : 9978
--------------------------------------------------
Completeness (%)                : 99.78%
--------------------------------------------------
Cardinality                     : 3
--------------------------------------------------
Uniqueness (%)                  : 0.03%
--------------------------------------------------
Distinctness (%)                : 0.03%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 3
--------------------------------------------------
Field Length (MAX)              : 6
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ STRESS LEVEL — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Medium                              3387       33.87%
Low                                 3320        33.2%
High                                3271       32.71%
                                      22        0.22%


####2.11 Heart Disease Status

In [70]:
analyze_column(df, 'Heart Disease Status')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — HEART DISEASE STATUS</span>

Field Name                      : Heart Disease Status
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xx
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

Total Rows                      : 10000
--------------------------------------------------
NULL Count                      : 0
--------------------------------------------------
Missing (blank strings)         : 0
--------------------------------------------------
Actual (Non-missing)            : 10000
--------------------------------------------------
Completeness (%)                : 100.00%
--------------------------------------------------
Cardinality                     : 2
--------------------------------------------------
Uniqueness (%)                  : 0.02%
--------------------------------------------------
Distinctness (%)                : 0.02%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types (Count)        : 1
--------------------------------------------------
Field Length (MIN)              : 2
--------------------------------------------------
Field Length (MAX)              : 3
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ HEART DISEASE STATUS — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
No                                  8000        80.0%
Yes                                 2000        20.0%
